# Imports


In [17]:
library(tidyverse)
library(data.table)

# Read data

In [18]:
df_raw = readxl::read_excel('./data/Dados (NOVO).xlsx')

head(df_raw)

numero,D_IRA,idade,cor,peso,altura,imc,has,dm,icc,...,uNa_uCr_media_D1_D5,UVNa24h_media_D1_D5,uOsm_media_D1_D5,uNa+uK_media_D1_D5,uOsm_estimada_media_D1_D5,FENa_media_D1_D5,FEK_media_D1_D5,sU_sCr_media_D1_D5,uU_sU_media_D1_D5,delta_sCr_ media_D1_D5
4,2,51,1,58.5,1.75,19.10204,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,2,82,1,72.0,1.75,23.51020,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
23,2,50,1,110.0,1.70,38.06228,1,1,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
26,2,79,1,46.0,1.55,19.14672,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
31,2,26,1,60.0,1.75,19.59184,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
46,2,54,2,55.0,1.65,20.20202,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [19]:
df_raw %>%
    select(`uK_D-1`, `uNa_D-1`, `uNa+uK_D-1`) %>%
    mutate(soma = `uK_D-1` + `uNa_D-1`) %>%
    head()

uK_D-1,uNa_D-1,uNa+uK_D-1,soma
26.0,107.0,133.0,133.0
88.0,23.0,111.0,111.0
60.5,18.2,78.7,78.7
39.3,96.8,136.1,136.1
115.3,21.9,137.2,137.2
60.9,62.5,123.4,123.4


In [24]:
categorical_columns = c('D_IRA', 'cor', 'has', 'dm', 'icc',
                        'asma_dpoc', 'hiv', 'ca_ativo', 'avc', 
                        'dvp', 'drogadicao', 'ventilacao_mecanica',
                        'diureticos', 'vasopressina', 'iSRAA',
                        'causa_IRA', 'KDIGO', 'criterio_IRA',
                        'pos_operatorio', 'mortalidade_intra_hospitalar')

daily_columns = c('diurese',
                  'bh',
                  'sofa',
                  'su',
                  'scr',
                  'sna',
                  'sk',
                  'sosm',
                  'uu',
                  'ucr',
                  'una',
                  'uk',
                  'volume_urinario',
                  'uvu24h',
                  'feu',
                  'uu_ucr',
                  'una_ucr',
                  'uvna24h',
                  'uosm',
                  'uosm_estimada',
                  'una+uk',
                  'uu_su',
                  'fena',
                  'fek',
                  'su_scr',
                  'delta_scr')

# Wrangling

In [25]:
columns = colnames(df_raw)

df = df_raw %>%
    mutate_at(categorical_columns, funs(factor(.))) %>%
    mutate(IRA = factor(ifelse(D_IRA != 0, 1, 0)),
           KDIGO = factor(KDIGO),
           KDIGO_agrupado = case_when(KDIGO %in% c(2, 3) ~ "MODERADO/GRAVE",
                                      KDIGO == 1 ~ "LEVE",
                                      KDIGO == 0 ~ "SEM IRA")) 

names(df) <- str_replace_all(tolower(names(df)), c("-" = "_", " " = ""))

df %>%
    head()

numero,d_ira,idade,cor,peso,altura,imc,has,dm,icc,...,uosm_media_d1_d5,una+uk_media_d1_d5,uosm_estimada_media_d1_d5,fena_media_d1_d5,fek_media_d1_d5,su_scr_media_d1_d5,uu_su_media_d1_d5,delta_scr_media_d1_d5,ira,kdigo_agrupado
4,2,51,1,58.5,1.75,19.10204,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
7,2,82,1,72.0,1.75,23.51020,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
23,2,50,1,110.0,1.70,38.06228,1,1,1,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
26,2,79,1,46.0,1.55,19.14672,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,MODERADO/GRAVE
31,2,26,1,60.0,1.75,19.59184,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE
46,2,54,2,55.0,1.65,20.20202,1,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,1,LEVE


In [26]:
names(df)

[1] "numero"                       "d_ira"                       
  [3] "idade"                        "cor"                         
  [5] "peso"                         "altura"                      
  [7] "imc"                          "has"                         
  [9] "dm"                           "icc"                         
 [11] "asma_dpoc"                    "hiv"                         
 [13] "ca_ativo"                     "avc"                         
 [15] "dvp"                          "drogadicao"                  
 [17] "score_clinico"                "ventilacao_mecanica"         
 [19] "dva_mcg_kg_min"               "diureticos"                  
 [21] "vasopressina"                 "israa"                       
 [23] "saps3"                        "egfr_basal"                  
 [25] "scr_basal"                    "causa_ira"                   
 [27] "kdigo"                        "criterio_ira"                
 [29] "pos_operatorio"               "mortalidade_intra_hospitalar"
 [31] "tempo_internacao_hospitalar"  "diurese_d_4"                 
 [33] "bh_d_4"                       "sofa_d_4"                    
 [35] "su_d_4"                       "scr_d_4"                     
 [37] "sna_d_4"                      "sk_d_4"                      
 [39] "sosm_d_4"                     "uu_d_4"                      
 [41] "ucr_d_4"                      "una_d_4"                     
 [43] "uk_d_4"                       "volume_urinario_d_4"         
 [45] "uvu24h_d_4"                   "feu_d_4"                     
 [47] "uu_ucr_d_4"                   "una_ucr_d_4"                 
 [49] "uvna24h_d_4"                  "uosm_d_4"                    
 [51] "una+uk_d_4"                   "uosm_estimada_d_4"           
 [53] "fena_d_4"                     "fek_d_4"                     
 [55] "su_scr_d_4"                   "uu_su_d_4"                   
 [57] "delta_scr_d_4"                "diurese_d_3"                 
 [59] "bh_d_3"                       "sofa_d_3"                    
 [61] "su_d_3"                       "scr_d_3"                     
 [63] "sna_d_3"                      "sk_d_3"                      
 [65] "sosm_d_3"                     "uu_d_3"                      
 [67] "ucr_d_3"                      "una_d_3"                     
 [69] "uk_d_3"                       "volume_urinario_d_3"         
 [71] "uvu24h_d_3"                   "feu_d_3"                     
 [73] "uu_ucr_d_3"                   "una_ucr_d_3"                 
 [75] "uvna24h_d_3"                  "uosm_d_3"                    
 [77] "una+uk_d_3"                   "uosm_estimada_d_3"           
 [79] "fena_d_3"                     "fek_d_3"                     
 [81] "su_scr_d_3"                   "uu_su_d_3"                   
 [83] "delta_scr_d_3"                "diurese_d_2"                 
 [85] "bh_d_2"                       "sofa_d_2"                    
 [87] "su_d_2"                       "scr_d_2"                     
 [89] "sna_d_2"                      "sk_d_2"                      
 [91] "sosm_d_2"                     "uu_d_2"                      
 [93] "ucr_d_2"                      "una_d_2"                     
 [95] "uk_d_2"                       "volume_urinario_d_2"         
 [97] "uvu24h_d_2"                   "feu_d_2"                     
 [99] "uu_ucr_d_2"                   "una_ucr_d_2"                 
[101] "uvna24h_d_2"                  "uosm_d_2"                    
[103] "una+uk_d_2"                   "uosm_estimada_d_2"           
[105] "fena_d_2"                     "fek_d_2"                     
[107] "su_scr_d_2"                   "uu_su_d_2"                   
[109] "delta_scr_d_2"                "diurese_d_1"                 
[111] "bh_d_1"                       "sofa_d_1"                    
[113] "su_d_1"                       "scr_d_1"                     
[115] "sna_d_1"                      "sk_d_1"                      
[117] "sosm_d_1"                     "uu_d_1"             

In [27]:
for (col in daily_columns){
    df = df %>%
        mutate(!!col := if_else(ira == 1, !!sym(paste0(col, '_d_1')), 
                                !!sym(paste0(col, '_media_d1_d5'))))
}

df %>% select(daily_columns)

diurese,bh,sofa,su,scr,sna,sk,sosm,uu,ucr,...,una_ucr,uvna24h,uosm,uosm_estimada,una+uk,uu_su,fena,fek,su_scr,delta_scr
0.8000000,932.0000,3.0000000,59.00000,1.7200000,144.0000,4.600000,297.8333,7.800000,0.8300000,...,128.915663,119.84000,483.0000,395.95147,133.0000,0.13220339,1.53982597,11.712939,34.30233,0.18000000
1.4400000,-670.0000,3.0000000,54.00000,0.6300000,127.0000,3.200000,263.0000,11.700000,1.5900000,...,14.465409,57.04000,598.0000,416.92720,111.0000,0.21666667,0.07175754,10.896226,85.71429,0.06000000
0.7196970,739.0000,3.0000000,133.00000,0.9300000,139.0000,4.500000,300.1667,9.940000,0.5420000,...,33.579336,34.58000,349.0000,323.00482,78.7000,0.07473684,0.22466750,23.068881,143.01075,0.22000000
1.2681159,228.0000,5.0000000,55.00000,0.6900000,135.0000,3.500000,279.1667,8.090000,0.7950000,...,121.761006,135.52000,367.0000,406.98299,136.1000,0.14709091,0.62233403,9.745553,79.71014,0.00000000
0.5555556,-82.0000,2.0000000,27.00000,0.9100000,141.0000,4.600000,286.5000,16.420000,2.1010000,...,10.423608,17.52000,669.0000,547.96450,137.2000,0.60814815,0.06727293,10.856424,29.67033,0.08000000
1.8181818,-314.0000,3.0000000,24.00000,0.6100000,147.0000,4.000000,298.0000,26.580000,1.0734000,...,58.226197,150.00000,899.0000,689.63461,123.4000,1.10750000,0.24161891,8.652180,39.34426,0.16000000
0.9493671,-891.0000,4.0000000,38.00000,0.7100000,129.0000,3.300000,264.3333,17.240000,0.8427000,...,87.575650,132.84000,592.0000,505.22606,109.0000,0.45368421,0.48200551,8.986986,53.52113,0.14000000
0.6944444,950.0000,3.0000000,18.00000,0.8400000,139.0000,4.600000,281.0000,3.342000,1.4250000,...,72.280702,103.00000,323.0000,397.67920,171.0000,0.18566667,0.43680424,8.713959,21.42857,0.29000000
0.5169562,476.0000,2.0000000,37.00000,0.9900000,141.0000,4.200000,288.1667,9.566000,0.6020000,...,20.598007,12.40000,211.0000,216.77381,28.7000,0.25854054,0.14462430,6.382297,37.37374,0.01000000
1.2500000,230.0000,4.0000000,21.00000,0.5800000,143.0000,5.000000,289.5000,15.193000,1.2180000,...,39.901478,116.64000,549.0000,452.32213,99.6000,0.72347619,0.16183816,4.857143,36.20690,0.00000000


In [29]:
df = df %>%
    select(-matches("d_?\\d"))

dim(df)

[1] 55 59

In [30]:
head(df)

numero,d_ira,idade,cor,peso,altura,imc,has,dm,icc,...,una_ucr,uvna24h,uosm,uosm_estimada,una+uk,uu_su,fena,fek,su_scr,delta_scr
4,2,51,1,58.5,1.75,19.10204,0,0,0,...,128.91566,119.84,483,395.9515,133.0,0.13220339,1.53982597,11.712939,34.30233,0.18
7,2,82,1,72.0,1.75,23.51020,1,0,0,...,14.46541,57.04,598,416.9272,111.0,0.21666667,0.07175754,10.896226,85.71429,0.06
23,2,50,1,110.0,1.70,38.06228,1,1,1,...,33.57934,34.58,349,323.0048,78.7,0.07473684,0.22466750,23.068881,143.01075,0.22
26,2,79,1,46.0,1.55,19.14672,0,0,0,...,121.76101,135.52,367,406.9830,136.1,0.14709091,0.62233403,9.745553,79.71014,0.00
31,2,26,1,60.0,1.75,19.59184,0,0,0,...,10.42361,17.52,669,547.9645,137.2,0.60814815,0.06727293,10.856424,29.67033,0.08
46,2,54,2,55.0,1.65,20.20202,1,0,0,...,58.22620,150.00,899,689.6346,123.4,1.10750000,0.24161891,8.652180,39.34426,0.16


In [31]:
names(df)

[1] "numero"                       "d_ira"                       
 [3] "idade"                        "cor"                         
 [5] "peso"                         "altura"                      
 [7] "imc"                          "has"                         
 [9] "dm"                           "icc"                         
[11] "asma_dpoc"                    "hiv"                         
[13] "ca_ativo"                     "avc"                         
[15] "dvp"                          "drogadicao"                  
[17] "score_clinico"                "ventilacao_mecanica"         
[19] "dva_mcg_kg_min"               "diureticos"                  
[21] "vasopressina"                 "israa"                       
[23] "saps3"                        "egfr_basal"                  
[25] "scr_basal"                    "causa_ira"                   
[27] "kdigo"                        "criterio_ira"                
[29] "pos_operatorio"               "mortalidade_intra_hospitalar"
[31] "tempo_internacao_hospitalar"  "ira"                         
[33] "kdigo_agrupado"               "diurese"                     
[35] "bh"                           "sofa"                        
[37] "su"                           "scr"                         
[39] "sna"                          "sk"                          
[41] "sosm"                         "uu"                          
[43] "ucr"                          "una"                         
[45] "uk"                           "volume_urinario"             
[47] "uvu24h"                       "feu"                         
[49] "uu_ucr"                       "una_ucr"                     
[51] "uvna24h"                      "uosm"                        
[53] "uosm_estimada"                "una+uk"                      
[55] "uu_su"                        "fena"                        
[57] "fek"                          "su_scr"                      
[59] "delta_scr"

In [32]:
write.csv(df, "./data/processed_data_mean.csv", row.names=FALSE, quote=FALSE) 

In [33]:
saveRDS(df, "./data/processed_data_mean.rds")